In [1]:
!pip -q install google-api-python-client google-auth google-auth-httplib2 google-auth-oauthlib


In [2]:
from google.colab import auth
auth.authenticate_user()

import google.auth
creds, _ = google.auth.default()


In [3]:
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

docs_service = build("docs", "v1", credentials=creds)


In [4]:
MARKDOWN_NOTES = r"""# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

---
Meeting recorded by: Sarah Chen
Duration: 45 minutes
"""


In [5]:
import re
from dataclasses import dataclass
from typing import List, Optional, Tuple

# -----------------------------
# Data model for parsed lines
# -----------------------------
@dataclass
class DocLine:
    text: str
    kind: str  # "h1", "h2", "h3", "bullet", "checkbox", "hr", "footer", "blank"
    level: int = 0  # indentation level for bullets/checkbox
    is_footer: bool = False


# -----------------------------
# Markdown parsing (minimal but effective for given notes)
# -----------------------------
def parse_markdown_to_lines(md: str) -> List[DocLine]:
    lines: List[DocLine] = []

    in_footer = False
    for raw in md.splitlines():
        line = raw.rstrip("\n")

        # Detect footer separator
        if line.strip() == "---":
            lines.append(DocLine(text="", kind="blank"))
            in_footer = True
            continue

        if not line.strip():
            lines.append(DocLine(text="", kind="blank"))
            continue

        if in_footer:
            lines.append(DocLine(text=line.strip(), kind="footer", is_footer=True))
            continue

        # Headings
        if line.startswith("# "):
            lines.append(DocLine(text=line[2:].strip(), kind="h1"))
            continue
        if line.startswith("## "):
            lines.append(DocLine(text=line[3:].strip(), kind="h2"))
            continue
        if line.startswith("### "):
            lines.append(DocLine(text=line[4:].strip(), kind="h3"))
            continue

        # Checkbox items: "- [ ] ..."
        m_cb = re.match(r"^(\s*)-\s+\[\s\]\s+(.*)$", line)
        if m_cb:
            indent_spaces = len(m_cb.group(1).replace("\t", "  "))
            level = indent_spaces // 2
            lines.append(DocLine(text=m_cb.group(2).strip(), kind="checkbox", level=level))
            continue

        # Bullets: "-", "*" with optional indentation
        m_b = re.match(r"^(\s*)[-*]\s+(.*)$", line)
        if m_b:
            indent_spaces = len(m_b.group(1).replace("\t", "  "))
            level = indent_spaces // 2
            lines.append(DocLine(text=m_b.group(2).strip(), kind="bullet", level=level))
            continue

        # Fallback as normal paragraph -> treat as bullet level 0 or plain line
        # For this task, keep as normal line (no bullets)
        lines.append(DocLine(text=line.strip(), kind="paragraph"))

    return lines


# -----------------------------
# Google Docs helpers
# -----------------------------
def create_google_doc(docs_service, title: str) -> str:
    try:
        doc = docs_service.documents().create(body={"title": title}).execute()
        return doc["documentId"]
    except HttpError as e:
        raise RuntimeError(f"Failed to create document: {e}")


def build_insert_and_index_map(doc_lines: List[DocLine]) -> Tuple[str, List[Tuple[int, int, DocLine]]]:
    """
    Returns:
      full_text: the text to insert (with \n)
      index_map: list of (start_index, end_index, DocLine) for each line in full_text
    Note: Google Docs body content starts at index 1.
    """
    full_text_parts = []
    index_map = []
    cursor = 1  # Docs start index

    for dl in doc_lines:
        text_line = dl.text
        # ensure each line ends with newline for paragraph separation
        to_add = text_line + "\n"
        full_text_parts.append(to_add)

        start = cursor
        end = cursor + len(text_line)  # exclude newline for styling convenience
        index_map.append((start, end, dl))

        cursor += len(to_add)

    return "".join(full_text_parts), index_map


def find_mentions_ranges(full_text: str, base_index: int = 1) -> List[Tuple[int, int]]:
    """
    Finds @mentions like @sarah and returns Docs index ranges.
    """
    ranges = []
    for m in re.finditer(r"@\w+", full_text):
        start = base_index + m.start()
        end = base_index + m.end()
        ranges.append((start, end))
    return ranges


def batch_update(docs_service, document_id: str, requests: List[dict]):
    if not requests:
        return
    try:
        docs_service.documents().batchUpdate(
            documentId=document_id,
            body={"requests": requests}
        ).execute()
    except HttpError as e:
        raise RuntimeError(f"Google Docs batchUpdate failed: {e}")


# -----------------------------
# Main conversion: markdown -> doc
# -----------------------------
def convert_markdown_to_gdoc(docs_service, markdown: str, doc_title: str = "Product Team Sync") -> str:
    doc_lines = parse_markdown_to_lines(markdown)

    # 1) Create document
    document_id = create_google_doc(docs_service, doc_title)

    # 2) Build insertion text + index map
    full_text, index_map = build_insert_and_index_map(doc_lines)

    # 3) Insert all text at start of body (index 1)
    requests = [
        {"insertText": {"location": {"index": 1}, "text": full_text}}
    ]

    # 4) Paragraph styles + bullets/checkbox
    # Indentation: 18pt per level (simple, readable)
    INDENT_PT = 18

    for start, end, dl in index_map:
        # Skip blank
        if dl.kind == "blank":
            continue

        # Headings
        if dl.kind in ("h1", "h2", "h3"):
            named_style = {"h1": "HEADING_1", "h2": "HEADING_2", "h3": "HEADING_3"}[dl.kind]
            requests.append({
                "updateParagraphStyle": {
                    "range": {"startIndex": start, "endIndex": end + 1},  # +1 include newline
                    "paragraphStyle": {"namedStyleType": named_style},
                    "fields": "namedStyleType"
                }
            })
            continue

        # Footer distinct style
        if dl.kind == "footer":
            # paragraph: italic + smaller + gray-ish
            requests.append({
                "updateTextStyle": {
                    "range": {"startIndex": start, "endIndex": end},
                    "textStyle": {
                        "italic": True,
                        "fontSize": {"magnitude": 10, "unit": "PT"},
                        "foregroundColor": {
                            "color": {"rgbColor": {"red": 0.45, "green": 0.45, "blue": 0.45}}
                        }
                    },
                    "fields": "italic,fontSize,foregroundColor"
                }
            })
            continue

        # Bullets
        if dl.kind in ("bullet", "checkbox"):
            # Apply bullets/checkbox preset to the paragraph range
            # Bullet presets: "BULLET_DISC_CIRCLE_SQUARE" and checkbox preset.
            bullet_preset = "BULLET_DISC_CIRCLE_SQUARE"
            if dl.kind == "checkbox":
                # This preset name works in many Docs API examples; if your environment rejects it,
                # swap to another checkbox preset supported by your Docs API version.
                bullet_preset = "BULLET_CHECKBOX"

            requests.append({
                "createParagraphBullets": {
                    "range": {"startIndex": start, "endIndex": end + 1},
                    "bulletPreset": bullet_preset
                }
            })

            # Indentation for nesting
            if dl.level > 0:
                requests.append({
                    "updateParagraphStyle": {
                        "range": {"startIndex": start, "endIndex": end + 1},
                        "paragraphStyle": {
                            "indentStart": {"magnitude": dl.level * INDENT_PT, "unit": "PT"}
                        },
                        "fields": "indentStart"
                    }
                })
            continue

        # Paragraph (plain)
        if dl.kind == "paragraph":
            # Keep as normal text (no special style)
            pass

    # 5) Mention styling across entire inserted text
    mention_ranges = find_mentions_ranges(full_text, base_index=1)
    for s, e in mention_ranges:
        requests.append({
            "updateTextStyle": {
                "range": {"startIndex": s, "endIndex": e},
                "textStyle": {
                    "bold": True,
                    "foregroundColor": {
                        "color": {"rgbColor": {"red": 0.12, "green": 0.45, "blue": 0.95}}
                    }
                },
                "fields": "bold,foregroundColor"
            }
        })

    # 6) Execute updates
    batch_update(docs_service, document_id, requests)

    return document_id


# -----------------------------
# Run it
# -----------------------------
doc_id = convert_markdown_to_gdoc(docs_service, MARKDOWN_NOTES, doc_title="Product Team Sync")
print("Created Doc ID:", doc_id)
print("Open it here (replace DOC_ID): https://docs.google.com/document/d/" + doc_id + "/edit")


Created Doc ID: 1UJuRuhwxSnxiIhB0Zs76NFPfQ6nczsC5AWh_fHqMQME
Open it here (replace DOC_ID): https://docs.google.com/document/d/1UJuRuhwxSnxiIhB0Zs76NFPfQ6nczsC5AWh_fHqMQME/edit
